In [2]:
import tensorflow as tf
import pandas as pd 
import numpy as np

dataset = pd.read_csv('./dataset/Iris.csv')

2025-01-30 05:38:20.854820: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 05:38:20.857554: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 05:38:20.865296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738215500.878899    9241 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738215500.883138    9241 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 05:38:20.897019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
dataset.shape

(150, 6)

In [4]:
dataset['Species'].value_count

AttributeError: 'Series' object has no attribute 'value_count'

In [5]:
dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
dataset = pd.get_dummies(dataset, columns=['Species'])
values = list(dataset.columns.values)


In [7]:
dataset

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,True,False,False
1,2,4.9,3.0,1.4,0.2,True,False,False
2,3,4.7,3.2,1.3,0.2,True,False,False
3,4,4.6,3.1,1.5,0.2,True,False,False
4,5,5.0,3.6,1.4,0.2,True,False,False
...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,False,False,True
146,147,6.3,2.5,5.0,1.9,False,False,True
147,148,6.5,3.0,5.2,2.0,False,False,True
148,149,6.2,3.4,5.4,2.3,False,False,True


In [8]:
# output data
y = dataset[values[-3:]]
y = np.array(y, dtype='float32')

# input data
x = dataset[values[1:-3]]
x = np.array(x, dtype='float32')

# shuffle data
indices = np.random.choice(len(x), len(x), replace=False)
x_values = x[indices]
y_values = y[indices]

# creating a test and train set
test_size = 10
x_test = x_values[-test_size:]
x_train = x_values[:-test_size]
y_test = y_values[-test_size:]
y_train = y_values[:-test_size]



In [17]:
#session
sess = tf.function()

#interval / epochs
interval = 50
epoch = 500

#initialize Variable
x_data = tf.constant([None, 4], dtype=tf.float32)
y_target = tf.constant(None, 3], dtype=tf.float32)

# Input neurons: 4
# Hidden neurons: 8
# output neurons: 3

hidden_layer_nodes = 8

# create variables for Neural Network layers
w1 = tf.Variable(tf.random_normal(shape=[4,hidden_layer_nodes])) # inputs -> hidden layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # one biases for each hidden node
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,3])) # hidden inputs -> 1 output
b2 = tf.Variable(tf.random_normal(shape=[3]))   # 1 bias for the output

# operations
hidden_output = tf.nn.relu(tf.add(tf.matmul(x_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# cost function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

# initialize variables
init = tf.global_variables_initializer()
sess.run(init)


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (2487632641.py, line 10)

In [ ]:
# training
for i in range(1, (epoch + 1)):
    sess.run(optimizer, feed_dict={x_data: x_train, y_target: y_train})
    if i % interval == 0:
        print('Epoch', i)
        print('Loss', sess.run(loss, feed_dict={x_data: x_train, y_target: y_train}))


In [ ]:
# prediction
prediction = tf.argmax(final_output, 1)
correct_prediction = tf.equal(prediction, tf.argmax(y_target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    